# TorQ: Delete Everything


In [1]:
import os
import subprocess
import boto3
import json
import datetime

import pykx as kx

from managed_kx import *
from env import *

# Cluster names and database
from clusters import *

SCALING_GROUP_NAME="SCALING_GROUP_torq"

VOLUME_NAME="SHARED_torq"

# Managed KX Databases
DB_NAME="finspace-database"

DBVIEW_NAME=f"{DB_NAME}_DBVIEW"

In [2]:
# triggers credential get
session=None

if AWS_ACCESS_KEY_ID is None:
    print("Using Defaults ...")
    # create AWS session: using access variables
    session = boto3.Session()
else:
    print("Using variables ...")
    session = boto3.Session(
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        aws_session_token=AWS_SESSION_TOKEN
    )

# create finspace client
client = session.client(service_name='finspace', endpoint_url=ENDPOINT_URL)

Using variables ...


## Delete all resources

In [3]:
print("Deleting Clusters...")

cdf = get_clusters(client, environmentId=ENV_ID)

all_clusters = [d['name'] for d in clusters if 'name' in d]
   
# Clusters
for c in all_clusters:
    print(f"\tDelete: {c}")
    try:
        client.delete_kx_cluster(environmentId=ENV_ID, clusterName=c)
        print(f"\tDeleted: {c}")
    except:
        print(f"\tAlready Deleted? {c}")
        next

print("** Deleted Clusters **")

# Dataview
print(f"Deleting Dataview: {DBVIEW_NAME}...")
try:
    client.delete_kx_dataview(environmentId=ENV_ID, databaseName=DB_NAME, dataviewName=DBVIEW_NAME)
    wait_for_dataview_status(client=client, environmentId=ENV_ID, databaseName=DB_NAME, dataviewName=DBVIEW_NAME, status='DELETED', show_wait=True)
    print("** Deleted View **")
except:
    pass

# Volume
print(f"Deleting Volume: {VOLUME_NAME}...")
try:
    client.delete_kx_volume(environmentId=ENV_ID, volumeName=VOLUME_NAME)
    wait_for_volume_status(client=client, environmentId=ENV_ID, volumeName=VOLUME_NAME, status='DELETED', show_wait=True)
    print("** Deleted Volume **")
except:
    print(f"Already Deleted? {VOLUME_NAME}...")
    pass

# Scaling Group
print(f"Deleting Scaling Group: {SCALING_GROUP_NAME}...")
try:
    client.delete_kx_scaling_group(environmentId=ENV_ID, scalingGroupName = SCALING_GROUP_NAME)
    wait_for_scaling_group_status(client=client, environmentId=ENV_ID, scalingGroupName = SCALING_GROUP_NAME, status='DELETED', show_wait=True)
    print("** Deleted Scaling Group **")
except:
    print(f"Already Deleted? {SCALING_GROUP_NAME}...")
    pass

# database
print(f"Deleting Database: {DB_NAME}...")
try:
    client.delete_kx_database(environmentId=ENV_ID, databaseName=DB_NAME)    
    print("** Deleted Database **")
except:
    pass

print("\nDone Deleting")

Deleting Clusters...
	Delete: discovery
	Deleted: discovery
	Delete: rdb
	Deleted: rdb
	Delete: hdb
	Deleted: hdb
	Delete: discovery
	Already Deleted? discovery
	Delete: rdb
	Already Deleted? rdb
	Delete: gateway
	Deleted: gateway
	Delete: feed
	Deleted: feed
** Deleted Clusters **
Deleting Dataview: finspace-database_DBVIEW...
Dataview: finspace-database_DBVIEW status is DELETING, total wait 0:00:00, waiting 30 sec ...
Dataview: finspace-database_DBVIEW status is DELETING, total wait 0:00:30, waiting 30 sec ...
Dataview: finspace-database_DBVIEW status is DELETING, total wait 0:01:00, waiting 30 sec ...
Dataview: finspace-database_DBVIEW status is DELETING, total wait 0:01:30, waiting 30 sec ...
Dataview: finspace-database_DBVIEW status is DELETING, total wait 0:02:00, waiting 30 sec ...
** Deleted View **
Deleting Volume: SHARED_torq...
** Deleted Volume **
Deleting Scaling Group: SCALING_GROUP_torq...
** Deleted Scaling Group **
Deleting Database: finspace-database...
** Deleted Dat

In [4]:
print( f"Last Run: {datetime.datetime.now()}" )

Last Run: 2024-03-01 20:37:15.962614
